# The project showcase an AI real estate agent built by state of the art LLM tools

In [1]:
import numpy as np #version 1.26.4
import pandas as pd #
import random
import openai
import re
from openai import OpenAI

In [2]:
with open('openai_api_key.txt', 'r') as file:
    openai_api_key = file.read()
client = OpenAI(api_key=openai_api_key)

In [27]:
property_types = ["condo", "apartment", "house", "ranch", "mansion"]

agent_prompt = """
You are a real estate agent, there are many properties on the market for sale, including condo, apartment, house, ranch, and mansion.
The listing must include: neighborhood name, property type, price, size, number of bedrooms, number of bathrooms, description, neighborhood description.
Use the following format:

Neighborhood name (be creative):
Property type:
Price:
Size:
Number of bedrooms:
Number of bathrooms:
Description:
Neighborhood Description (be creative):

Generate a description for a {} listed for sale:
"""

buyer_prompt = """
You are a real estate buyer, you are interetsed in buying a {}.
A real estate agent is helping you choosing a property listing that best suits your preferences.

Answer the following questions:

Questions:
What kind of property are you interested in buying?
What kind of neighborhood would you like to live in?
How many bedrooms and bathrooms do you need?
What amenities would you like?

Answers:
"""

## Step 1: Use OpenAI GPT-3.5-Turbo to generate listings of properties

In [28]:
def generate_listing_description(agent_prompt):
    """
    Using custom prompt to generate property listings
    from gpt-3.5-turbo model. 
    """
    try:
        response = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": agent_prompt,
                }
            ],
            model="gpt-3.5-turbo",
            temperature = 0.9
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(e)
        return ""

def generate_listings(num_listings, agent_prompt, property_types):
    """
    generate a number (num_listings) of listings using generate_listing_description
    parse the response string into a dictionary
    storm the listing dictionaries into a Pandas dataframe
    """
    listings = []
    success = False
    keys = ["neighborhood name",
            "property type",
            "price",
            "size",
            "number of bedrooms",
            "number of bathrooms",
            "description",
            "neighborhood description"]
    for i in range(num_listings):
        property_type = property_types[int(np.mod(i,5))]
        prompt = agent_prompt.format(property_type)
        listing_str = generate_listing_description(prompt).lower()
        listing_dict = {key: "" for key in keys}
        current_key = None
        for line in listing_str.split('\n'):
            line_key = next((key for key in keys if line.startswith(key + ":")), None)
            if line_key:
                current_key = line_key
                listing_dict[current_key] = line.split(": ", 1)[1].strip()
            elif current_key:
                listing_dict[current_key] += " " + line.strip()
        listings.append(listing_dict)
    listings_df = pd.DataFrame(listings)
    return listings_df

In [41]:
# Test generate_listings
num_listings = 20
listings_df = generate_listings(num_listings, agent_prompt, property_types)

In [45]:
listings_df.head(20)

,neighborhood name,property type,price,size,number of bedrooms,number of bathrooms,description,neighborhood description
0,pinecrest pines,condo,"$250,000",900 sq ft,2,2,this cozy condo in pinecrest pines offers a co...,pinecrest pines is a peaceful community surrou...
1,willowbrook heights,apartment,"$300,000",900 sq. ft.,2,1,this cozy and modern apartment is perfect for ...,willowbrook heights is known for its tree-line...
2,willow creek estates,house,"$500,000","2,500 sq ft",4,3,this stunning two-story house in willow creek ...,"willow creek estates is a charming, tree-lined..."
3,rolling hills retreat,ranch,"$1,200,000",10 acres,4,3,this stunning ranch features a spacious 4 bedr...,rolling hills retreat is a peaceful and seclud...
4,willowbrook estates,mansion,"$5,000,000","10,000 square feet",7,9,this luxurious mansion features high-end finis...,willowbrook estates is an exclusive enclave of...
5,serene heights,condo,"$300,000",900 sqft,2,2,this modern and stylish condo in serene height...,serene heights is a bustling neighborhood know...
6,sunset heights,apartment,"$300,000","1,200 sqft",2,1.5,this cozy apartment in sunset heights offers c...,sunset heights is a vibrant neighborhood known...
7,willow creek estates,house,"$750,000","3,500 square feet",4,3.5,this beautiful 2-story house in willow creek e...,willow creek estates is a quiet and family-fri...
8,sunset valley ranch,ranch,"$1,200,000",20 acres,4,3,enjoy the quiet serenity of sunset valley ranc...,sunset valley ranch is located in the heart of...
9,diamond hills estates,mansion,"$5,000,000","10,000 square feet",5,7,this luxurious mansion in diamond hills estate...,diamond hills estates is known for its exclusi...


In [46]:
listings_df.to_csv("listings.csv")

## Step 2: Create embeddings for the listings

In [47]:
# load saved listing data
# listings_df = pd.read_csv("./listings.csv", index_col=0)

In [48]:
# concatenate description and neighborhood description into one column
listings_df["combined_description"] = listings_df["description"].str.cat(listings_df["neighborhood description"], sep=" ")

In [49]:
# Using "text-embedding-ada-002" as text embedding model
def get_embedding(text):
    text = text.replace("\n", " ")
    model = "text-embedding-ada-002"
    return client.embeddings.create(input=[text], model=model).data[0].embedding

# Add embeddings list to dataframe
listings_df["ada_embeddings"] = listings_df["combined_description"].apply(get_embedding)

In [50]:
# convert price from string to float
# listings_df["price"] = listings_df["price"].str.replace("$", "").str.replace(",", "").astype(float)
listings_df.head()

,neighborhood name,property type,price,size,number of bedrooms,number of bathrooms,description,neighborhood description,combined_description,ada_embeddings
0,pinecrest pines,condo,"$250,000",900 sq ft,2,2,this cozy condo in pinecrest pines offers a co...,pinecrest pines is a peaceful community surrou...,this cozy condo in pinecrest pines offers a co...,"[0.019364628940820694, -0.0077577559277415276,..."
1,willowbrook heights,apartment,"$300,000",900 sq. ft.,2,1,this cozy and modern apartment is perfect for ...,willowbrook heights is known for its tree-line...,this cozy and modern apartment is perfect for ...,"[-0.006473578047007322, -0.0033602677285671234..."
2,willow creek estates,house,"$500,000","2,500 sq ft",4,3,this stunning two-story house in willow creek ...,"willow creek estates is a charming, tree-lined...",this stunning two-story house in willow creek ...,"[-0.003830644767731428, -0.004575492348521948,..."
3,rolling hills retreat,ranch,"$1,200,000",10 acres,4,3,this stunning ranch features a spacious 4 bedr...,rolling hills retreat is a peaceful and seclud...,this stunning ranch features a spacious 4 bedr...,"[-0.00853109173476696, 0.014227195642888546, -..."
4,willowbrook estates,mansion,"$5,000,000","10,000 square feet",7,9,this luxurious mansion features high-end finis...,willowbrook estates is an exclusive enclave of...,this luxurious mansion features high-end finis...,"[-0.015052882954478264, -0.003955600783228874,..."


In [51]:
# save listings_df with embeddings
listings_df.to_csv("./listings_with_embedding.csv")

## Step 3: Store listings in a vector database

In [52]:
import lancedb
from lancedb.pydantic import vector, LanceModel

class PropertyListings(LanceModel):
    neighborhood_name: str
    property_type: str
    price: str
    size: str
    num_bedrooms: int
    num_bathrooms: float
    description: str
    neighborhood_description: str
    combined_description: str
    ada_embeddings: vector(1536)

db = lancedb.connect("./.lancedb")
table_name = "property_listings"
db.drop_table(table_name)
table = db.create_table(table_name, schema = PropertyListings)

In [53]:
listings_df.rename(columns={"neighborhood name": "neighborhood_name",
                            "property type": "property_type",
                            "number of bedrooms": "num_bedrooms",
                            "number of bathrooms": "num_bathrooms",
                            "neighborhood description": "neighborhood_description"
                           }, inplace=True)
table.add(listings_df)

# Step 4: Collect buyer preferences

In [54]:
def generate_buyer_preference():
    """
    Using custom prompt to generate property listings
    from gpt-3.5-turbo model. 
    """

    try:
        response = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": buyer_prompt.format(random.choice(property_types)),
                }
            ],
            model="gpt-3.5-turbo",
            temperature = 0.5
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(e)
        return ""
def clean_buyer_preference(response):
    if "\n\n" in response:
        response = response.split("\n\n")
    if "\n" in response:
        response = response.split("\n")
    if "?" in response:
        response = [s.strip() for s in response if "?" not in s]
    result_response = " ".join(response)
    # remove numerical bullet points in result_response
    result_response = re.sub(r'\d+\. ', '', result_response)
    # remove "-" in result_response
    result_response = re.sub(r'- ', '', result_response)
    # remove "I [text] " pattern
    cleaned_response = re.sub(r'I \w+ ', '', result_response)
    return cleaned_response

def search_listings(table, preference, num:int):
    """
    inputs:
        listings_df (Pandas DataFrame): listings database
        query (str): customer preference
        num (int): number of recommendations
    outputs:
        reco_df (Pandas DataFrame): recommendations 
    
    """
    cleaned_preference = clean_buyer_preference(preference)
    embedding = get_embedding(cleaned_preference)
    reco_df = table.search(embedding).limit(num).to_pandas()
    return reco_df

In [55]:
preference = generate_buyer_preference()
preference

'I am interested in buying a condo.\nI would like to live in a neighborhood that is safe, has good schools, and is close to amenities such as restaurants, shopping centers, and parks.\nI would need at least 2 bedrooms and 2 bathrooms.\nI would like amenities such as a gym, swimming pool, and a community garden.'

In [56]:
cleaned_preference = clean_buyer_preference(preference)
cleaned_preference

'interested in buying a condo. like to live in a neighborhood that is safe, has good schools, and is close to amenities such as restaurants, shopping centers, and parks. need at least 2 bedrooms and 2 bathrooms. like amenities such as a gym, swimming pool, and a community garden.'

# Step 5: Search listings based on buyer's preference

## Test a buyer

In [64]:
sample_preference = generate_buyer_preference()
print(sample_preference)
reco_df = search_listings(table, sample_preference, 3)
reco_df

I am interested in buying a single-family home.
I would like to live in a safe and family-friendly neighborhood with good schools.
I would need at least 3 bedrooms and 2 bathrooms.
I would like amenities such as a backyard, a garage, and a modern kitchen.


,neighborhood_name,property_type,price,size,num_bedrooms,num_bathrooms,description,neighborhood_description,combined_description,ada_embeddings,_distance
0,serene meadows,house,"$500,000","2,500 sq ft",4,2.5,"this charming 4 bedroom, 2.5 bathroom house in...",serene meadows is a quiet and family-friendly ...,"this charming 4 bedroom, 2.5 bathroom house in...","[-0.0007424627, 0.027851487, -0.018284183, -0....",0.343752
1,willowbrook heights,apartment,"$300,000",900 sq. ft.,2,1.0,this cozy and modern apartment is perfect for ...,willowbrook heights is known for its tree-line...,this cozy and modern apartment is perfect for ...,"[-0.006473578, -0.0033602677, 0.016648225, -0....",0.368013
2,serene haven heights,apartment,"$300,000",900 sq ft,2,1.0,this charming apartment is perfect for those l...,serene haven heights is a quiet and family-fri...,this charming apartment is perfect for those l...,"[0.01952567, 0.0137843825, 0.010258914, -0.002...",0.369944


# Step 6: Alter the retrieved listing's description with the buyer's preferences

## Use Retreival Augmented Generation (RAG) technique to alter the description of the matched property

In [65]:
rag_prompt = """
You are a real estate agent.
Genearte a tailored description based on the context below, highlight the specific preferences in the context.
Do not change factual information including name, neighborhood, amenities, and location.

Context: 

{}

---

Description: 

{}

Tailored description:
"""

In [66]:
def generate_custom_listing_description(preference, num):
    reco_df = search_listings(table, preference, num)
    agent_reco = generate_listing_description(rag_prompt.format(preference, reco_df.iloc[0]["combined_description"]))
    reco_df.drop(['combined_description', 'ada_embeddings', '_distance'], axis=1, inplace=True)
    return agent_reco, reco_df

In [67]:
n = 3
agent_reco, reco_df = generate_custom_listing_description(preference, n)
print(agent_reco)
reco_df.head()

Welcome to your dream home in Serene Meadows! This beautiful 4 bedroom, 2.5 bathroom house is perfect for those seeking a safe and family-friendly neighborhood with top-rated schools. Enjoy the modern kitchen with stainless steel appliances, a cozy fireplace in the spacious living room, and a backyard with a patio for outdoor gatherings. The master bedroom features a walk-in closet and en-suite bathroom for added convenience. With easy access to parks, hiking trails, and shopping centers, this home offers the perfect blend of comfort and convenience for you and your family. Don't miss out on the opportunity to make Serene Meadows your new home!


,neighborhood_name,property_type,price,size,num_bedrooms,num_bathrooms,description,neighborhood_description
0,serene meadows,house,"$500,000","2,500 sq ft",4,2.5,"this charming 4 bedroom, 2.5 bathroom house in...",serene meadows is a quiet and family-friendly ...
1,willowbrook heights,apartment,"$300,000",900 sq. ft.,2,1.0,this cozy and modern apartment is perfect for ...,willowbrook heights is known for its tree-line...
2,serene haven heights,apartment,"$300,000",900 sq ft,2,1.0,this charming apartment is perfect for those l...,serene haven heights is a quiet and family-fri...


# Step 7: Implement user interface as app

In [68]:
import gradio as gr

In [76]:
def update_results_label(value):
    label = f"Showing Top {value} Results"
    return gr.Dataframe(reco_df, label=label, show_label=True)

with gr.Blocks() as demo:
    with gr.Row():
        client_pref = gr.Textbox(value=sample_preference, label="Client Preference",show_label=True)
    with gr.Row():
        b1 = gr.Button("Generate Client Preferences")
        b2 = gr.Button("Submit")
        num = gr.Slider(minimum=1, maximum=10, label="Number of Results", step=1)
    with gr.Row():
        agent_reco = gr.Textbox(value="", label="Agent Recommendation",show_label=True)
    with gr.Row():
        reco_list = gr.Dataframe(reco_df, label=f"Showing Top {num.value} Results", show_label=True)
    with gr.Row():
        avail_listings = gr.Dataframe(listings_df, label="Available Listings", show_label=True)
    num.change(update_results_label, inputs=num, outputs=reco_list)
    b1.click(generate_buyer_preference, outputs=client_pref)
    b2.click(generate_custom_listing_description, inputs=[client_pref, num], outputs=[agent_reco, reco_list])

demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [77]:
demo.close()

Closing server running on port: 7861


In [81]:
from importlib.metadata import version
package_list = ["numpy", "pandas", "openai", "lancedb", "gradio"]
numpy_version = version('numpy')
pandas_version = version('pandas')
openai_version = version('openai')
lancedb_version = version('lancedb')
gradio_version = version('gradio')
versions_str = ""
for package in package_list:
    versions_str = versions_str + package + " version: " + version(package) + "\n"

with open("package_version_requirements.txt", 'w') as file:
    file.write(versions_str)